In [1]:
import os

# Нарезаем видос на кадры
videos_path = './test_data/'

for video_name in os.listdir(videos_path):
    if 'mkv' not in video_name:
        continue
    new_folder = 'frames-' + video_name.split('.mkv')[0]
    os.makedirs(os.path.join(videos_path, new_folder), exist_ok=True)
    inp = os.path.join(videos_path, video_name)
    out = os.path.join(videos_path, new_folder, '%04d.png')
    !ffmpeg -i {inp} -r 1/1 {out}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [2]:
import os
from shadow_models import *

data_root = './test_data/frames-2023-12-23-00-16-19/'
save_dir_masks = './outputs_masks'
save_dir_restored = './outputs_restored'
os.makedirs(save_dir_masks, exist_ok=True)
os.makedirs(save_dir_restored, exist_ok=True)

model_restoration, model_detection = get_shadow_models()

You choose ShadowFormer...


/home/andrey/anaconda3/envs/shadow/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loaded pretrained weights for efficientnet-b3


In [3]:
test_dataset = ShadowDataset(data_root=data_root, im_size=512)

In [4]:
# full_image - полное разрешение
# image - scale до im_size=512 в детекторе
# все в numpy от 0 до 255
test_dataset[0].keys()

odict_keys(['full_image', 'image', 'im_shape', 'im_name'])

In [5]:
with torch.no_grad():
    for data in tqdm(test_dataset):
        
        im_name = data['im_name']
        
        # refine вроде не существенно влияет на качество, но сильно замедляет
        detector_output = run_detector(model_detection, data, refine=False)
        rgb_restored = run_shadowformer(model_restoration, data, detector_output)
        
        cv2.imwrite(os.path.join(save_dir_masks, im_name), detector_output)
        cv2.imwrite(os.path.join(save_dir_restored, im_name), rgb_restored)

100%|█████████████████████████████████████████████████████████████████████████████████| 127/127 [00:32<00:00,  3.92it/s]


In [8]:
!ffmpeg -framerate 10 -i "./outputs_restored/%04d.png" output.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab